In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine


In [2]:
# Paths
covid_crime_data = pd.read_csv('../Output/clean_covid_crime_data.csv')
income_data = pd.read_csv('../Output/clean_income_data.csv')
employment_data = pd.read_csv('../Output/employement_clean.csv')


In [3]:
# Postgres connection info
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'
database = 'covid_crime_db'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

In [4]:
# Connect to default database
default_engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/postgres')

In [5]:
# Create a new database
try:
    # Connect to the database
    conn = default_engine.connect()
    
    # Create new database
    conn.execute("commit")
    conn.execute("CREATE DATABASE covid_crime_db")
    
    # Close the connection
    conn.close()
except OperationalError as e:
    print(f"Error occurred: {e}")

# Now connect to your newly created database
new_db_engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/covid_crime_db')

In [6]:
# import contacts to sql
covid_crime_data.to_sql("crime", con=engine.connect(), if_exists='replace')
income_data.to_sql("income", con=engine.connect(), if_exists='replace')
employment_data.to_sql("employment", con=engine.connect(), if_exists='replace')

840

In [7]:
with engine.connect() as con:
    con.execute('ALTER TABLE crime ADD PRIMARY KEY (index);')
    con.execute('ALTER TABLE income ADD PRIMARY KEY (index);')
    con.execute('ALTER TABLE employment ADD PRIMARY KEY (index);')